<a href="https://colab.research.google.com/github/yaroslavtsepkov/APC/blob/main/monte_carlo/pi_monte_carlo_numpy_and_pycuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

In [ ]:
!pip install scikit-cuda

In [ ]:
import numpy as np
import time
import pandas as pd
import cupy as cp
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from  pycuda import  driver
from pycuda import cumath
from pycuda.compiler import SourceModule
from pycuda.curandom import rand as curand
import matplotlib.pyplot as plt


In [ ]:
def genData(p):
    x_gpu = curand((p,), dtype=np.double) 
    y_gpu = curand((p,), dtype=np.double)
    x = x_gpu.get().astype(np.float)
    y = y_gpu.get().astype(np.float)
    return x, y, p

In [ ]:
def piCalcNumpy(x, y, p):
    values = x ** 2 + y ** 2 < 1
    return 4.0/p * values[values == True].shape[0]

In [ ]:
def piCalcCupy(x, y, p):
    x, y = cp.array(x), cp.array(y)
    values = cp.power(x, 2) + cp.power(y, 2) < 1
    return 4.0/p * values[values == True].shape[0]

In [ ]:
def piCalcPyCUDA(x, y, p):
    x, y = gpuarray.to_gpu(x), gpuarray.to_gpu(y)
    values = x ** 2 + y ** 2 < 1
    return float((4.0 / p * gpuarray.sum(values)).get())

In [ ]:
points = [1024, 2048, 4096, 65536, 16777216]

In [ ]:
numpytime, numpyresult = [],[]
cupytime, cupyresult = [], []
pycudatime, pycudaresult = [],[]
pycudatime_kernal, pycudaresult_kernal = [],[]

In [ ]:
for p in points:
    x, y, p = genData(p)
    t=time.time()
    numpyresult.append(piCalcNumpy(x,y,p))
    numpytime.append(time.time()-t)
    t=time.time()
    cupyresult.append(piCalcCupy(x,y,p))
    cupytime.append(time.time()-t)
    t=time.time()
    pycudaresult.append(piCalcPyCUDA(x,y,p))
    pycudatime.append(time.time()-t)
 

In [ ]:
mod = SourceModule("""
                __global__ void gpu_pi_monte_carlo(double *x, double *y, double *count) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x; 
        int threadCount = gridDim.x * blockDim.x;
        int p = 16777216;
        int count_gpu = 0;
        for (int i = idx; i < p; i += threadCount) {
                if (x[i] * x[i] + y[i] * y[i] < 1) {
                        count_gpu++;
                }
        }
        atomicAdd(count , count_gpu);
}
""")

In [ ]:
gpu_pi_monte_carlo = mod.get_function("gpu_pi_monte_carlo")
t = time.time()
gpu_result = gpuarray.zeros((1,), dtype=np.double)
gpu_result  = gpu_result.get()
gpu_pi_monte_carlo(driver.In(x), driver.In(y),  driver.Out(gpu_result), np.int32(p), block = (128, 1, 1), grid =(int(p/(128 * 128)), 1))
driver.Context.synchronize()
pycudatime_kernal.append(time.time()-t)
pycudaresult_kernal.append(gpu_result[0] * 4/p)

In [ ]:
data={'numpyresult':numpyresult,'cupyresult':cupyresult,'pycudaresult':pycudaresult,'pycudaresult_kernal':pycudaresult_kernal}

In [ ]:
data

In [ ]:
data_time={'numpytime':numpytime,'cupytime':cupytime,'pycudatime':pycudatime,'pycudatime_kernal':pycudatime_kernal}
data_time

In [ ]:
pd.DataFrame(data = data, index=points, )

In [ ]:
pd.DataFrame(data = data_time, index=points, )

In [ ]:
data_time.keys()

In [ ]:
for i in data_time:
    print(data_time[i])

In [ ]:
df.to_csv()